# Display Wannier Orbitals

## I/O Functions

In [57]:
import numpy as np
from tvtk.api import tvtk
from mayavi.scripts import mayavi2

def import_xsf(filenames):
    '''
    This functions reads a 3D dataset form a xsf file and returns the corresponding vtk structured grid data-set
    
    Input:      filname - Name of the xsf file
    
    Return:     sgrid - vtkStructuredGrid instance containing the 3D dataset
                        or null if no 3D data set is contained in the file
    '''
    
    def read_tokens(f):
        '''
            Reads space separated values from a text file going across lines
            
            Usage:
                tokens = read_tokens(f)
                value1 = next(tokens)
                value2 = next(tokens)
                ...
        '''
        for line in f:
            for token in line.split():
                yield token
            
    def read_3d_datagrid(f):
        '''
            Read a BLOCK_DATAGRID_3D block from a XSF file.
            
            Input:    f - file object positioned after the BEGIN_BLOCK_DATAGRID_3D line
            
            Returns:  dims    - integer array of dimension 3 containing the number of points in all directions
                      points  - cartesian coordinates of all grid points in FORTRAN order (x axis has fastest index)
                      values  - volume data a points coordinates            
        '''
        
        # Skip next two lines
        f.readline()
        f.readline()

        # Read dataset size
        l = [int(a) for a in f.readline().split()]
        nvalues = l[0]*l[1]*l[2]

        # Read geometry 
        origin = [float(a) for a in f.readline().split()]
        lattice_a = np.array([float(a) for a in f.readline().split()])
        lattice_b = np.array([float(a) for a in f.readline().split()])
        lattice_c = np.array([float(a) for a in f.readline().split()])

        data = np.zeros(nvalues)
        tokens = read_tokens(f)
        # Read volume data
        for x in range(nvalues):
            data[x]= float(next(tokens))

        # Prepare coordinate arrays
        points = np.empty([nvalues,3])
        for k in range(l[2]):
            for j in range(l[1]):
                for i in range(l[0]):
                    idx = i + j*l[0] + k*l[0]*l[1]
                    points[idx,:] = i*lattice_a/l[0] + j*lattice_b/l[1] + k*lattice_c/l[2] + origin

         # Make the data.
        dims = (l[0], l[1], l[2])

        return dims,points,data
    
    global sum_values 
    sum_values = None
    
    for fn in filenames:
        
        print('Processing {}'.format(fn))
        
        # Process entire file and extract data
        with open(fn) as f:
            
            sgrid = None

            for line in f:
                
                if line.strip() == 'BEGIN_BLOCK_DATAGRID_3D': 
                    dims, points, values = read_3d_datagrid(f)
                    
                    if sum_values is None:
                        sum_values = np.absolute(values)
                    else:
                        sum_values = sum_values + np.absolute(values)
                    
    sgrid = tvtk.StructuredGrid(dimensions=dims)
    sgrid.points = points

    sgrid.point_data.scalars = sum_values
    sgrid.point_data.scalars.name = 'scalars'

                    
    return sgrid


# Visualise Orbitals

In [63]:
filenames = ['wannier90.dn_{:05n}.xsf'.format(i+1) for i in range(112,120)]
sgrid = import_xsf(filenames)
w = tvtk.XMLStructuredGridWriter(file_name="orbital_O-pxz.vts")
w.set_input_data(sgrid)
w.write()

Processing wannier90.dn_00113.xsf
Processing wannier90.dn_00114.xsf
Processing wannier90.dn_00115.xsf
Processing wannier90.dn_00116.xsf
Processing wannier90.dn_00117.xsf
Processing wannier90.dn_00118.xsf
Processing wannier90.dn_00119.xsf
Processing wannier90.dn_00120.xsf


1